In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
r = requests.get("https://cnwoman-bot.github.io/evil-man/")
soup = BeautifulSoup(r.text,'html.parser')
soup.prettify()


In [ ]:
soup

## getting crimes list

In [24]:
def get_news_within(div):
    p = div.next_sibling.next_sibling
    lst_text = [a.text for a in p.find_all('a')]
    return lst_text
    

In [25]:
text = [i.parent.text.split('\n') for i in soup.find_all('a') if i.text.startswith('判决日期')]
o = []
for t in text:
    if len(t) <= 3:
        o.append(t[0])
    else:
        for s in t:
            if '判决日期' not in s:
                o.append(s)
o = list(set(o))
other1 = o

other2 = ['新疆喀什大学文学院援疆博士王明科','云南云南大学文学院副教授蔡英杰性骚扰留学生被公开处分，现任福建师范大学教授','甘肃兰州大学大气科学学院副院长张文煜被曝性侵',
         '上海戏剧学院教务处处长厉震林性骚扰，已无链接','央美设计学院院长宋协伟被指控性骚扰、猥亵女学生', '原北京中央民族大学文学院与新闻传播学院刘立刚性骚扰',
         '湖北美院偷拍女生裙底把视频发到了p站，无语，链接不放了','北京电影学院阿廖莎性侵案','北京电影学院侯亮平事件']

In [26]:
div = [d for d in soup.find_all('div',class_="language-plaintext highlighter-rouge") if '本小节合计' in d.text]
crime = []
for d in div:
    crime += get_news_within(d)
crime += other1
crime += other2
l = ['亳州6·15杀人案,邻居眼中的老实人,为何连杀4人','四川一名大学本科男子刘某在成都开往深圳的火车上猥亵9岁女童','浙大学生努某某强奸被判一年半，学校不与开除，留校察看','猥亵女童的网红“豆浆王子”蒙顺宁，出狱后获得公益组织爱心帮扶',
 '西南某大学法学院22岁男生偷拍女生裙底',
 '陕西安康石泉30岁的男子索爱一女子不成跳江自杀，女子被判决赔偿各项损失七万元']
crime += l
out1 = [i for i in crime if "洋媳妇" in i][0]
out2 = [i for i in crime if "所有的功劳就还是男性的" in i][0]
out2
crime.remove(out1)
crime.remove(out2)


In [ ]:
#some crimes are nested under h3 tags not div - but really messy

## getting cities dct

In [ ]:
#still need another scraping to get a list of cities

In [5]:
#getting the city list
import requests
from bs4 import BeautifulSoup
import re

r_city = requests.get("https://en.wikipedia.org/wiki/List_of_cities_in_China")
s_city = BeautifulSoup(r_city.text,'html.parser')
s_city.prettify()

import chinese_converter

def divide(string):
    string = chinese_converter.to_simplified(string)
    if '特别行政区' in string: 
        return string[0:2]
    else:
        return string[:-1]

dct_city = {}
normal_cities = s_city.find_all('tr',bgcolor="Lavender")
for c in normal_cities:
    lst = c.text.split()
    cn_name = lst[1]
    cn_name = divide(cn_name)
    dct_city[cn_name] = lst[0]
    
special_cities = s_city.find_all('tr',bgcolor="Thistle")+ s_city.find_all('tr',bgcolor="Moccasin")
for c in special_cities:
    txt = re.sub(' ','',c.text) #this is for the special case of HK
    lst = txt.split()
    cn_name = lst[1]
    cn_name = divide(cn_name)
    dct_city[cn_name] = lst[0]

In [ ]:
from china_cities import *
#this package returns all cities on different levels (e.g. municipal vs counties), 
#hence we don't use cities from this package;we only use the province part
province = [re.findall(r'\w+',re.sub(' ','',p).lower())[0] for p in cities.get_provinces()]
province

In [ ]:
counties = {i[:-1] for i in cities.get_cities_cn()} - set(dct_city.keys())
counties

## matching crimes to cities

In [211]:
dct_news = {}
lst_matched = []
for c in dct_city.keys():
    c_lst = [n for n in crime if c in n]
    dct_news[c] = len(c_lst)
    lst_matched += c_lst

In [212]:
len(lst_matched)

1419

## dealing with left out

In [ ]:
lst_left = [n for n in crime if n not in lst_matched]
lst_left

In [62]:
import jieba
import pinyin

In [250]:
#this is still flawed
#requesting the link of the news to get city info
def into_link(string):
    url = [a for a in soup.find_all('a') if string in a.text][0]['href']
    if 'http' not in url:
        url = 'http://' + url
    rr = requests.get(url)
    ss = BeautifulSoup(rr.text,'html.parser')
    ss.text.split()
    if 'forbidden' in ss.text.lower() or 'error' in ss.text.lower() or '删除' in ss.text.lower() or '找不到' in ss.text.lower():
        lst_no_loc.remove(string)
    else:
        loc_info = [t for t in set(jieba.cut(s4.text)) if '市' in t]
        if len(loc_info) == 1:
            city = loc_info[0]
            if city in dct_city.keys():
                dct_news[city] += 1
                lst_matched.append(string)
                lst_no_loc.remove(string)
            else:
                prov = [c.province for c in cities.get_cities() if c.name_cn == city]
                if len(prov) == 1:
                    p = prov[0]
                    county_owner(p, city)
                    lst_no_loc.remove(string)
                else:
                    return False
        else:
            return False

In [251]:
for l in lst_no_loc:
    into_link(l)  

InvalidURL: Invalid URL 'http://': No host supplied

In [200]:
def has_location(string):
    seg_list = jieba.cut(string)
    words = list(seg_list)
    fw = pinyin.get(words[0], format="strip", delimiter="")
    if fw in province and words[1] in counties:
        return fw, words[1]
    else:
        return None

In [209]:
def county_owner(fw, county_name):
    county_tag = [tag for tag in s_city.find_all('span',title="Chinese-language text") if county_name in tag.text][0]
    owner_pinyin = [t for t in county_tag.parent.parent.next_siblings][3].text  
    owner_city_lst = [c.name_cn for c in cities.get_cities() if fw in c.province.lower() and c.name_en == owner_pinyin]
    if len(owner_city_lst) != 1:
        if 'none' not in owner_pinyin:
            return False
        else:
            owner_city = county_name
    else:
        owner_city = owner_city_lst[0][:-1]
        
    if owner_city in dct_news.keys(): #because of the special case of enshi, which is not a municipal city but has no governing city
        dct_news[owner_city] += 1
    else:
        dct_news[owner_city] = 1        

In [ ]:
lst_no_loc = []
for l in lst_left:
    if has_location(l) != None:
        prov = has_location(l)[0]
        county = has_location(l)[1]
        if county_owner(prov, county) != False:
            lst_matched.append(l)
        else:
            lst_no_loc.append(l)
    else:
        lst_no_loc.append(l)

In [214]:
dct_news.keys()
len(lst_matched)

1479

In [ ]:
#earlier codes for fun:

In [259]:
set_weap = {'刀','棍','硫酸','斧头','锄头','榔头','墨'}
dct_w = {}
for w in set_weap:
    lst_crime = [n for n in crime if w in n]
    dct_w[w] = len(lst_crime)
dct_w

{'墨': 2, '榔头': 0, '斧头': 3, '刀': 163, '硫酸': 2, '锄头': 1, '棍': 7}

In [257]:
set_crime = {'杀害','猥亵','强奸','强暴','家暴','殴打','偷拍','偷窥','性侵','毒','药','埋','溺','推','勒','碎尸','毒','药','烧'}
dct_c = {}
for c in set_crime:
    lst_crime = [n for n in crime if c in n]
    dct_c[c] = len(lst_crime)
dct_c

{'强暴': 10,
 '杀害': 183,
 '猥亵': 281,
 '性侵': 152,
 '强奸': 179,
 '家暴': 30,
 '溺': 2,
 '毒': 18,
 '偷拍': 70,
 '烧': 21,
 '殴打': 50,
 '勒': 20,
 '推': 9,
 '碎尸': 10,
 '埋': 14,
 '药': 23,
 '偷窥': 12}

In [119]:
[n for n in news if '女德' in n]

['女德班成果：“中国女孩只属于中国男孩”', '广东东莞蒙正国学馆“女德班” 无证办学被责令停办，曾称要做女强人就得切除子宫乳房']

In [249]:
rr = requests.get('https://baijiahao.baidu.com/s?id=1622347467847141868&wfr=spider&for=pc')
s4 = BeautifulSoup(rr.text,'html.parser')
s4.prettify()
[t for t in set(jieba.cut(s4.text)) if '省' in t]

[]